# Can you SEE MY SCREEN?

<span style="color: red;"> 
How do we connect LLMs with Tools?
</span>


In [1]:
from openai import OpenAI
import os

client = OpenAI()

# if you don't have an API key as an environment variable you can set it here

# os.environ["OPENAI_API_KEY"] = ""

In [2]:
def llm_response(prompt: str):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

llm_response("How do we connect LLMs with Tools?")

"Connecting Large Language Models (LLMs) with external tools involves integrating the model's output with various APIs, scripts, databases, or other software functionalities to enhance its capabilities. Here are some key steps and considerations for effectively connecting LLMs with tools:\n\n### 1. **Identify Use Cases**:\n   - Determine the specific needs that can be met by integrating tools with an LLM. Common use cases include data retrieval, performing calculations, sending emails, accessing calendars, or executing code.\n\n### 2. **Define Interfaces**:\n   - **API Integration**: Many tools expose APIs that can be called by your application. LLMs can be programmed to formulate requests based on their output.\n   - **Function Calls**: You can create functions that take inputs from the LLM’s output and execute specific tasks or calculations.\n\n### 3. **Setting Up the Environment**:\n   - Ensure that your development environment has access to both the LLM and the tools you want to co

We could ask the model to:

1. Generate Python code and then give the model the ability to run that code somewhere
2. Create the Python function that executes the task, then somehow give the model the ability to run that function autonomously

Option number 2 is what we call FUNCTION CALLING!

# Function Calling from Scratch

1. Write a Python function
2. Make that function "available" to the LLM model of choice (in this case will be gpt-4o/gpt-4o-mini)
3. Test if the model can execute that function properly
   1. Prepare the payload for the function (the arguments for it...)
   2. Write the function call
   3. We run the function call ourselves
   4. We inspect the results

In [3]:
def create_folder(folder_path: str):
    os.makedirs(folder_path, exist_ok=True)
    return f"Folder created at {folder_path}"

create_folder("./pancakes-are-the-best")

'Folder created at ./pancakes-are-the-best'

In [7]:
!ls -d ./* | grep pancakes

./pancakes-are-the-best


In [4]:
def write_file(file_path: str, content: str):
    with open(file_path, "w") as file:
        file.write(content)
    return f"File created at {file_path}"

write_file("./pancakes-are-the-best/pancakes-are-the-best.txt", "Pancakes are the best!")

'File created at ./pancakes-are-the-best/pancakes-are-the-best.txt'

In [9]:
!cat pancakes-are-the-best/pancakes-are-the-best.txt

Pancakes are the best!

# LLM + Python Functions

The most silly way possible first!

In [5]:
prompt_with_function_information = """
You are a personal assistant with desktop capabilities. 
Users will ask you to perform tasks and you will execute those tasks by calling one or more of the following functions:

- create_folder(folder_path: str)
- write_file(file_path: str, content: str)

For example if the user asks: 

'Create a folder called lucas-teaches-function-callling'

Your output should be code like this:
'create_folder("./lucas-teaches-function-callling")'

'Create a folder called pancakes-are-the-best and write a file called pancakes-are-the-best.txt with the content "Pancakes are the best!"'

Your output should be code like this:

['create_folder("./pancakes-are-the-best")', 'write_file("./pancakes-are-the-best/pancakes-are-the-best.txt", "Pancakes are the best!")']

Your OUTPUT should ALWAYS BE python code only.

Here is the user prompt:
"""


user_prompt = "Create a file named pancakes-are-better-than-waffles.md with a one paragraph summary for why pancakes are better, \
make sure to address RS a student from my course who dared to say avocado waffles are better."

prompt = prompt_with_function_information + user_prompt

llm_response(prompt)

'```python\nwrite_file("./pancakes-are-better-than-waffles.md", "Pancakes are better than waffles because they are fluffy, soft, and can be made in various flavors, making them versatile for any breakfast craving. Unlike avocado waffles, which may be perceived as overly complex or niche, pancakes offer a classic taste that pleases everyone, including you, RS. Whether topped with syrup, fruit, or whipped cream, pancakes bring comfort and joy that avocado waffles simply can\'t match.")\n```'

In [13]:

output_with_python_code_and_ticks = llm_response(prompt)

output_with_python_code_and_ticks

'```python\nwrite_file("./pancakes-are-better-than-waffles.md", "Pancakes are better than waffles for many reasons, RS. Unlike waffles, which can often be too crispy and dry, pancakes offer a soft, fluffy texture that\'s perfect for soaking up syrup or fruit toppings. Additionally, pancakes can be made in countless flavors, from classic buttermilk to banana or chocolate chip, giving them a versatility that waffles simply can\'t match. So next time you think about breakfast options, remember that pancakes have a charm and comfort that can easily surpass avocado waffles!")\n```'

Let's clean up this output a little bit:

In [15]:
output_cleaned = output_with_python_code_and_ticks.replace("```python\n", "").replace("```", "")

output_cleaned

'write_file("./pancakes-are-better-than-waffles.md", "Pancakes are better than waffles for many reasons, RS. Unlike waffles, which can often be too crispy and dry, pancakes offer a soft, fluffy texture that\'s perfect for soaking up syrup or fruit toppings. Additionally, pancakes can be made in countless flavors, from classic buttermilk to banana or chocolate chip, giving them a versatility that waffles simply can\'t match. So next time you think about breakfast options, remember that pancakes have a charm and comfort that can easily surpass avocado waffles!")\n'

Let's find a way to execute this code!

In [16]:
exec(output_cleaned)

In [17]:
!ls | grep pancakes

pancakes-are-better-than-waffles.md
pancakes-are-the-best


We could improve this a little bit by writing up a couple of functions that do a slightly better job at cleaning up the output generated by the LLM,
and then executing the resulting function call:

In [8]:
import re

def parse_llm_function_calls(llm_output: str):
    """
    Naive parser that looks for function calls in the model output (e.g. 'create_folder("test")')
    and returns a list of dicts, each containing 'function_name' and 'args'.
    """
    calls = []
    
    # Regex looks for function_name( ... ) 
    # capturing everything until the next matching parenthesis
    pattern = r"(\w+)\(([^)]*)\)"
    matches = re.findall(pattern, llm_output)
    
    for match in matches:
        function_name = match[0]
        raw_args = match[1].split(',')
        
        # Clean up quotes/spaces
        parsed_args = [arg.strip().strip('"').strip("'") for arg in raw_args]
        calls.append({
            "function_name": function_name,
            "args": parsed_args
        })
    
    return calls

def execute_llm_function_calls(calls):
    """
    Executes each call by mapping the function_name to an actual Python function.
    If a function is unknown, handle gracefully.
    """
    for call in calls:
        fn_name = call["function_name"]
        args = call["args"]
        
        if fn_name == "create_folder":
            create_folder(*args)
        elif fn_name == "write_file":
            write_file(*args)
        else:
            print(f"[Warning] Unknown function: {fn_name}")

# Example usage
# Suppose the model returned the following text (as a single string):
# 'create_folder("./some_folder") write_file("./some_folder/readme.txt","Some content")'

model_generated_code = """
create_folder("./some_folder")
write_file("./some_folder/readme.txt", "Some content")
"""

# 1) Parse
calls = parse_llm_function_calls(model_generated_code)

# 2) Execute
execute_llm_function_calls(calls)

In [11]:
!ls some_folder/

readme.txt


We connected LLM + FUNCTION via some simple PROMPT magic!

In [10]:
!ls -d */ | grep some

some_folder/
